<a href="https://colab.research.google.com/github/goodScienceRice/Open-Assistant/blob/main/Knowledge_Embedding_Community_Blog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Pinecone

### Connect Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

###Installing Pinecone Packages

In [ ]:
!pip install pandas==1.5.3
!pip install -qU pinecone-client pandas

In [ ]:
import pinecone

###Installing Other required packages

In [ ]:
!pip install openai
!pip install langchain
!pip install tiktoken

In [ ]:
import os
import openai
OPENAI_API_KEY=''
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
openai.api_key = os.getenv("OPENAI_API_KEY")

from tqdm.auto import tqdm
from time import sleep
from uuid import uuid4
import tiktoken

###Pinecone keys configuration

In [ ]:
pinecone.init(api_key="", environment="")

###Pinecone Index details

In [ ]:
pinecone.list_indexes()

['knowledge']

In [ ]:
pinecone.Index('knowledge').describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 17}},
 'total_vector_count': 17}

In [ ]:
#pinecone.delete_index("knowledge")

###PDF, CSV, TXT Files are Supported

PDF Upload

In [ ]:
!pip install PyPDF2
import PyPDF2

In [ ]:
# importing required modules
from PyPDF2 import PdfReader

# creating a pdf reader object
reader = PdfReader('stevejobs.pdf') #Replace 'stevejobs.pdf' with your .pdf file

# getting a specific page from the pdf file
page = reader.pages

In [ ]:
text = ""
for page in reader.pages:
   text+=page.extract_text()
#print(type(text), len(text))
print(text)

CSV Upload



In [ ]:
# Replace 'Pinecone test - Sheet1.csv' with your .csv file
import csv

filename = 'Pinecone test - Sheet1.csv'

# If you want to store lines in a list:
lines_string = ""
with open(filename, 'r') as csv_file:
    reader = csv.reader(csv_file)
    for row in reader:
        lines_string += ' '.join(row) + "\n"

text = lines_string
print(text)

TXT Upload

In [ ]:
# Replace 'stevejobs.txt' with your filename
with open('stevejobs.txt') as f:
    text = f.read()

###Text Chunking

In [ ]:
from langchain.text_splitter import TokenTextSplitter
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=10)
input_chunks = text_splitter.split_text(text)
i=0
for item in input_chunks:
  i+=1
  print(f'\nChunk {i}:\n')
  print(item)


In [ ]:
len(input_chunks)

189

###Embedding Model and Batch Size

In [ ]:
embed_model = "text-embedding-ada-002"
batch_size=6

###Creating Embedding

In [ ]:
def create_embeddings(texts):
  res = openai.Embedding.create(input=texts, engine=embed_model)
  for record in res['data']:
    embeds = record['embedding']
  return embeds

In [ ]:
final_chunks=[]
final_chunks.extend([{
   "id": str(uuid4()),
   "text": input_chunks[i],
   "chunk": i,
   "knowledge_name": "knowledge", # Replace "knowledge" with the your index name
   "embeds" : create_embeddings(input_chunks[i]),
} for i in range(len(input_chunks))])

In [ ]:
len(final_chunks)

###Pinecone Index creation and Upsert

In [ ]:
pinecone.create_index("knowledge", dimension=1536, metric="dotproduct") # Replace "knowledge" with the your index name

In [ ]:
index = pinecone.Index('knowledge') # Replace "knowledge" with the your index name

In [ ]:
for i in tqdm(range(0, len(final_chunks), batch_size)):
      # find end of batch
  i_end = min(len(final_chunks), i + batch_size) #1
  meta_batch = final_chunks[i:i_end]
  batch_id = [x['id'] for x in meta_batch]
      # get texts to encode
  texts = [x['text'] for x in meta_batch]
      # create embeddings (try-except added to avoid RateLimitError)
  try:
    res = openai.Embedding.create(input=texts, engine=embed_model)
  except:
    done = False
    while not done:
      sleep(5)
      try:
        res = openai.Embedding.create(input=texts, engine=embed_model)
        done = True
      except:
        pass
  embeds = [record['embedding'] for record in res['data']]
      # cleanup metadata
  meta_batch = [{
    'text': x['text'],
    'chunk': x['chunk'],
    'knowledge_name': x['knowledge_name'], # Replace "knowledge" with the your index name
  } for x in meta_batch]
  to_upsert = list(zip(batch_id, embeds, meta_batch))
  print(to_upsert)
      # upsert to Pinecone
  index.upsert(vectors=to_upsert)



In [ ]:
pinecone.Index('knowledge').describe_index_stats() # Replace "knowledge" with the your index name

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 206}},
 'total_vector_count': 206}